In [1]:
import pandas as pd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

C:\Users\Bismar\.conda\envs\agro-cittca\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Load the private key for SSH


FileNotFoundError: [Errno 2] No such file or directory: 'llave'

In [5]:
# CONECTAR A AMIGOCLOUD
data = open('../api_key/agrocittca.data','r')
password = data.readlines()[0]
data.close()

In [14]:
password

"{'ssh_host': '167.99.184.211', 'ssh_port': 22, 'localhost': '127.0.0.1', 'localport': 3306, 'ssh_user': 'forge', 'sql_username': 'forge', 'sql_password': '21q5DVqO9tbCQjnQkNnO',\n"

In [ ]:
def dbQuery(q):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [ ]:
D = dbQuery('select * from users where estado=\'activo\'')

In [ ]:
df = pd.DataFrame(D)

In [11]:
datos = {
    'ssh_host' : '167.99.184.211',
    'ssh_port' : 22,
    'localhost' : '127.0.0.1',
    'localport' : 3306,
    'ssh_user' : 'forge',
    'sql_username' : 'forge',
    'sql_password' : '21q5DVqO9tbCQjnQkNnO',
    'sql_main_database' : 'guabira'
}

In [12]:
datos

{'ssh_host': '167.99.184.211',
 'ssh_port': 22,
 'localhost': '127.0.0.1',
 'localport': 3306,
 'ssh_user': 'forge',
 'sql_username': 'forge',
 'sql_password': '21q5DVqO9tbCQjnQkNnO',
 'sql_main_database': 'guabira'}